In [1]:
import cv2
import numpy as np

import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
from tkinter import messagebox as mb
from tkinter.filedialog import asksaveasfile

import PIL
from PIL import Image, ImageTk

import os

# import keyboard
# from pynput.keyboard import Key, Listener

# Dark images apply threshold

# GrabCut Foreground Extraction, cut and paste onto other images

# Haar Cascade Object Detection Face & Eye 

# User Interface

In [7]:
import tkinter as tk                
from tkinter import font  as tkfont 

class SampleApp(tk.Tk):

    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)

        self.title_font = tkfont.Font(family='Times', size=18, weight="bold", slant="italic")
        self.subtitle_font = tkfont.Font(family='Times', size=10, weight="bold", slant="italic")

        # the container is where we'll stack a bunch of frames
        # on top of each other, then the one we want visible
        # will be raised above the others
        container = tk.Frame(self)
        container.pack(side="top", fill="both", expand=True)
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)

        self.frames = {}
        for F in (StartPage, PageOne, PageTwo, PageThree):
            page_name = F.__name__
            frame = F(parent=container, controller=self)
            self.frames[page_name] = frame

            # put all of the pages in the same location;
            # the one on the top of the stacking order
            # will be the one that is visible.
            frame.grid(row=0, column=0, sticky="nsew")

        self.show_frame("StartPage")

    def show_frame(self, page_name):
        '''Show a frame for the given page name'''
        frame = self.frames[page_name]
        frame.tkraise()                

class StartPage(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        label = tk.Label(self, text="Welcome to my Mini Project\n\nPlease Choose The Following Operations:\n\n", font=controller.title_font)
        label.pack(side="top", fill="x", pady=10)
        
        btn_thresh = tk.Button(self, text="Threshold (Gaussian)",command=lambda: controller.show_frame("PageOne"))        
        btn_grabcut = tk.Button(self, text="Grabcut Foreground Extraction",command=lambda: controller.show_frame("PageTwo"))
        btn_haar = tk.Button(self, text="Haar Cascade",command=lambda: controller.show_frame("PageThree"))
        btn_quit = tk.Button(self, text="Quit", bg="red" ,command=lambda: controller.destroy())

        btn_thresh.pack(pady=5)        
        btn_grabcut.pack(pady=5)
        btn_haar.pack(pady=5)
        btn_quit.pack(pady=30)
        
# Threshold
class PageOne(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
                
        label = tk.Label(self, text="Threshold (Gaussian)", font=controller.title_font)
        label.pack(side="top", fill="x", pady=10)
                
        btn_browse = tk.Button(self, text = "Browse A File", command=self.fileDialog)
        btn_return = tk.Button(self, text="Return to menu", fg="blue", command=lambda: controller.show_frame("StartPage"))
                
        btn_return.pack()
        btn_browse.pack() 
        
        
    def fileDialog(self):
                
        self.filename = filedialog.askopenfilename(initialdir =  "/home/biozxel/Desktop/", 
                                                   title = "Select A Image", 
                                                   filetypes =(("jpeg files","*.jpg"),("png files","*.png*")))        

        load = Image.open(self.filename)         
        render = ImageTk.PhotoImage(load)
        
        img = Label(self, image=render)        
        img.image = render
        img.place(x=150, y=150)        

        self.applyThreshold(self.filename)

    def applyThreshold(self, filename): 
        
        print(filename) 
        fr = cv2.imread(filename)
                            
        retval, threshold = cv2.threshold(fr, 12, 255, cv2.THRESH_BINARY)

        grayscaled = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
        gaus = cv2.adaptiveThreshold(grayscaled, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY, 115, 1)
            
        imgtk = ImageTk.PhotoImage(image=PIL.Image.fromarray(gaus))
            
        img = Label(self, image=imgtk)
        img.image = imgtk
        img.place(x=150, y=500)
        
#         file = asksaveasfile(mode="w",filetypes =(("jpeg files","*.jpg"),("png files","*.png*")))
#         cv2.imwrite(file, imgtk) 
        
# Grabcut            
class PageTwo(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        label = tk.Label(self, text="GrabCut Foreground Extraction", font=controller.title_font)
        label.pack(side="top", fill="x", pady=10)
        
        btn_return = tk.Button(self, text="Return to menu", fg="blue",command=lambda: controller.show_frame("StartPage"))
        btn_browse = tk.Button(self, text = "Browse A File", command=self.fileDialog)

        btn_return.pack()
        btn_browse.pack()
        
    def fileDialog(self):
        self.filename = filedialog.askopenfilename(initialdir =  "/home/biozxel/Desktop/", 
                                                   title = "Select A Image", 
                                                   filetypes =(("jpeg files","*.jpg"),("png files","*.png*")))        

        load = Image.open(self.filename)        
        render = ImageTk.PhotoImage(load)
        
        print(self.filename)
        
        img = Label(self, image=render)
        img.image = render
        img.place(x=150, y=500)
        
        self.applyGrabcut(self.filename)
        
    def store_images(self, image):        
        print(image)        
   
        pic_num = 0

        if not os.path.exists('grabcut-images'):
            os.makedirs('grabcut-images')
            mb.showinfo("folder created","folder 'grabcut-images' has been created to store the saved images")
            
        pic_num += 1            
        print(pic_num)
        
        cv2.imwrite("grabcut-image-"+str(pic_num)+".jpg",np.float32(image))

        mb.showinfo("Success","Image successfully saved")

    def applyGrabcut(self, filename):
        
        print(filename)        

        img4 = cv2.imread(filename, cv2.IMREAD_COLOR)
        mask = np.zeros(img4.shape[:2], np.uint8)

        # background foreground model
        bgdModel = np.zeros((1,65), np.float64)
        fgdModel = np.zeros((1,65), np.float64)

        # for foreground
        rect = (39,31,310,298)

        # test for certain regions
        rect_test = (59,27,313,228)
        rect_test2 = (200,100,900,45)

        # grab and cut certain region (GC --> GrabCut)
        cv2.grabCut(img4, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)
        mask2 = np.where((mask==2) | (mask==0), 0, 1).astype('uint8') # 0 or 2 | 1 - 3 other options

        img4 = img4 * mask2[:,:,np.newaxis]

        imgtk = ImageTk.PhotoImage(image=PIL.Image.fromarray(img4))
        img = Label(self, image=imgtk)
        img.image = imgtk
        img.place(x=150, y=150)
        
#         self.store_images(imgtk)
                    
# Haar cascade        
class PageThree(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        label = tk.Label(self, text="Haar Cascade", font=controller.title_font)
        label.pack(side="top", fill="x", pady=10)
        
        btn_return = tk.Button(self, text="Return to menu", fg="blue",
                           command=lambda: controller.show_frame("StartPage"))
        
        btn_webcam = tk.Button(self, text="Open Webcam",
                           command=self.openWebcam)
                
        btn_browse = tk.Button(self, text="Browse an Image",
                           command=self.fileDialog)
        btn_return.pack()
        btn_browse.pack()
        btn_webcam.pack()        
            
    def openWebcam(self):
        face = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        eye = cv2.CascadeClassifier('haarcascade_eye.xml')
        
        cap = cv2.VideoCapture(0)
    
        if cap is None or not cap.isOpened():
            mb.showerror("Webcam Error","Webcam has not been turned on or there are issues with your webcam")
            print('Warning: unable to open video source: ', cap)
        else:
            mb.showinfo("To Stop Webcam","Press any key to stop the webcam")            
            cap = cv2.VideoCapture(0)
            
            hat = cv2.imread('cut-hat-transparent-1.jpg')
            img2grey = cv2.cvtColor(hat, cv2.COLOR_BGR2GRAY)

            blur = cv2.GaussianBlur(img2grey, (5,5), 0)
            ret, mask = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

            mask_inv = cv2.bitwise_not(mask) 

            _,alpha = cv2.threshold(img2grey,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            b, g, r = cv2.split(hat)
            rgba = [b,g,r, alpha]
            dst = cv2.merge(rgba,4)
            
            while True:  
                cap = cv2.VideoCapture(0)
                
                ret, vid = cap.read()

                gray = cv2.cvtColor(vid, cv2.COLOR_BGR2GRAY)
                faces = face.detectMultiScale(gray, 1.3, 5)

                for (x,y,w,h) in faces:
                    cv2.rectangle(vid, (x,y), (x+w, y+h), (255,0,0), 2)

                    rows, cols, channels = hat.shape
                    roi = vid[0:rows, 0:cols]

                    face_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
                    hat_fg = cv2.bitwise_and(hat, hat, mask=mask)

                    dst = cv2.add(face_bg, hat_fg)
                    vid[0:rows, 0:cols] = dst        

                    roi_gray = gray[y:y+h, x:x+w]
                    roi_colour = vid[y:y+h, x:x+w]

                    roi_hat = img2grey[y:y+h, x:x+w]

                    eyes = eye.detectMultiScale(roi_gray)        
                    for(ex,ey,ew,eh) in eyes:
                        cv2.rectangle(roi_colour, (ex,ey), (ex+ew, ey+eh), (0,255,0), 2)

                    cv2.imshow('vid', vid)
                    
                if cv2.waitKey(10) & 0XFF == ord('q'):
                    break
                    
            cap.release()
            cv2.destroyAllWindows()
                

    def fileDialog(self):
        self.filename = filedialog.askopenfilename(initialdir =  "/home/biozxel/Desktop/", 
                                                   title = "Select A Image", 
                                                   filetypes =(("jpeg files","*.jpg"),("png files","*.png*")))

        load = Image.open(self.filename)        
        render = ImageTk.PhotoImage(load)
        
        print(self.filename)
        
        img = Label(self, image=render)
        img.image = render
        img.place(x=150, y=150)
        
        self.applyHaar(self.filename)
        
    def applyHaar(self, filename):
        face = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        eye = cv2.CascadeClassifier('haarcascade_eye.xml')

        file = cv2.imread(filename)

        # convert to gray
        gray = cv2.cvtColor(file, cv2.COLOR_BGR2GRAY)
        faces = face.detectMultiScale(gray, 1.3, 5)

        # draw rectangle (x --> width, y --> height) to blue width of 2
        for (x,y,w,h) in faces:
            cv2.rectangle(file, (x,y), (x+w, y+h), (255,0,0), 2)

            roi_gray = gray[y:y+h, x:x+w]
            roi_colour = file[y:y+h, x:x+w] # reimpose colour

            # find eye within region of face
            eyes = eye.detectMultiScale(roi_gray)        
            for(ex,ey,ew,eh) in eyes:
                cv2.rectangle(roi_colour, (ex,ey), (ex+ew, ey+eh), (0,255,0), 2)
                
        imgtk = ImageTk.PhotoImage(image=PIL.Image.fromarray(file))
        img = Label(self, image=imgtk)
        img.image = imgtk
        img.place(x=150, y=500)        
        
if __name__ == "__main__":
    app = SampleApp()
    app.wm_geometry("700x400")
    app.mainloop()

/home/biozxel/Desktop/OpenCV/bookpage.jpg
/home/biozxel/Desktop/OpenCV/george-washington-face.jpg
/home/biozxel/Desktop/OpenCV/george-washington-face.jpg
/home/biozxel/Desktop/OpenCV/george-washington-face.jpg
